##  Borrowers’ risk of defaulting 

We'll prepare a report for a bank’s loan division. We need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

The report can be considered when building a **credit scoring** of a potential customer. A **credit scoring** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer

In [2]:
scoring = pd.read_csv('credit_scoring_eng.csv')

In [3]:
scoring.info()
scoring.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


### Conclusion

The file contains **21525** entries and **12** columns, **2** columns out of **12** though have missing values, specifically they're *days_employed* and *total_income* columns. 

Additionally those two columns have *float* type though we don't need such precise details for the analysis and can convert them into *int* type later.
*days_employed* column also contains negative values which should be examined below.

### Step 2. Data preprocessing

### Processing missing values
First, let's examine missing values and try to find any patterns. For **quantative** variables we need to replace missing values with some *represantative* value.

To find the patterns let's see if *days_employed* and *total_income* columns depend on each other or other columns when they contain NaN. For this purpose let's simply count the number of rows with NaN values in a certain column and compare the results.


In [2]:
# 1. Count the number of rows with NaN in the days_employed column
days_employed_nan = scoring[(scoring['days_employed'].isna())]['debt'].count()

# 2. Count the number of rows with NaN in the total_income column
total_income_nan = scoring[(scoring['total_income'].isna())]['debt'].count()

# 3. Count the number of rows with NaN simoultaneously in the days_employed and total_income columns
both_nan = scoring[(scoring['days_employed'].isna()) & (scoring['total_income'].isna())]['debt'].count()

# 4. Compare the results: if all three values are the same then NaN values occur simultaneously
if days_employed_nan == total_income_nan and days_employed_nan == both_nan:
    print('NaN values are always present in both columns simultaneously. The number of rows with missing values:', both_nan)
else:
    print('NaN values distibuted between the two columns independetly. The number of rows with missing values:', (days_employed_nan + total_income_nan - both_nan) )



NaN values are always present in both columns simultaneously. The number of rows with missing values: 2174


In [3]:

# 5. Now let's see if "NaN rows" contain any specific values in other columns.
print('=======================')
print('Children distribution:')
print(scoring[(scoring['days_employed'].isna())]['children'].value_counts())
print('=======================')
print('Education distribution')
print(scoring[(scoring['days_employed'].isna())]['education_id'].value_counts())
print('=======================')
print('Family status distribution')
print(scoring[(scoring['days_employed'].isna())]['family_status_id'].value_counts())
print('=======================')
print('Gender distribution')
print(scoring[(scoring['days_employed'].isna())]['gender'].value_counts())
print('=======================')
print('Income type distribution')
print(scoring[(scoring['days_employed'].isna())]['income_type'].value_counts())
print('=======================')
print('Purpose distribution')
print(scoring[(scoring['days_employed'].isna())]['purpose'].value_counts())
print('=======================')
print('Debt distribution:')
print(scoring[(scoring['days_employed'].isna())]['debt'].value_counts())
print('=======================')


Children distribution:
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64
Education distribution
1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64
Family status distribution
0    1237
1     442
4     288
3     112
2      95
Name: family_status_id, dtype: int64
Gender distribution
F    1484
M     690
Name: gender, dtype: int64
Income type distribution
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64
Purpose distribution
having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                             72
transactions with my real estate            71
purchase of the house for my family         71
transaction

## Results:

Noticed typos in children column (e.g the value of 20) but nothing special in values distribution. 

Let's examine _days_employed_ column. We saw that it contains positive and negative values. 
What's the range of each type?

In [4]:
# 6. What's the range of negative and positive values?
print("Positive values")
print(scoring[(scoring['days_employed']>0)].agg({'days_employed':['min', 'mean', 'median','max']}))
print("\nNegative values")
print(scoring[(scoring['days_employed']<=0)].agg({'days_employed':['min', 'mean', 'median', 'max']}))

Positive values
        days_employed
min     328728.720605
mean    365004.309916
median  365213.306266
max     401755.400475

Negative values
        days_employed
min     -18388.949901
mean     -2353.015932
median   -1630.019381
max        -24.141633


## Results:

It seems that positive values do not relate to reality as 365 000 days mean 1 000 years of work. 
Looks like a default infinity value. On the other hand absolute value of negatives contains relevant values: max 50 years, average 7 years.

Let's see if positive values have specific values in other columns. 

In [5]:
# 7. Let's see if positive values have specific values in other columns. 

scoring[ (scoring['days_employed']>=0) ].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
35,0,394021.072184,68,secondary education,1,civil partnership,1,M,retiree,0,12448.908,having a wedding
50,0,353731.432338,63,secondary education,1,married,0,F,retiree,0,14774.837,cars
56,0,370145.087237,64,secondary education,1,widow / widower,2,F,retiree,0,23862.567,education
71,0,338113.529892,62,secondary education,1,married,0,F,retiree,0,7028.751,cars
78,0,359722.945074,61,bachelor's degree,0,married,0,M,retiree,0,28020.423,purchase of a car


## Results

Noticed income_type 'retiree' relates to positive values. Let's check if it's the only value:

In [6]:
# 8. Let's check if it's the only value:

scoring[ (scoring['days_employed']>=0) ]['income_type'].value_counts()

retiree       3443
unemployed       2
Name: income_type, dtype: int64

## Results

So mostly all positive values mean 'retiree'. Let's look if negatives can relate to them.

In [7]:
# 9. Let's look if negatives can relate to them.
scoring[ (scoring['days_employed']<0) & (scoring['income_type']=='retiree')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [8]:
# snapshot of retiree with NaN before the replacement
scoring[ (scoring['days_employed'].isna()) & (scoring['income_type']=='retiree')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
145,0,NaN,62,secondary education,1,married,0,M,retiree,0,NaN,building a property
...,...,...,...,...,...,...,...,...,...,...,...,...
21311,0,NaN,49,secondary education,1,married,0,F,retiree,0,NaN,buying property for renting out
21321,0,NaN,56,Secondary Education,1,married,0,F,retiree,0,NaN,real estate transactions
21414,0,NaN,65,secondary education,1,married,0,F,retiree,0,NaN,purchase of my own house
21415,0,NaN,54,secondary education,1,married,0,F,retiree,0,NaN,housing transactions


## Results:

It means that retiree always imply positive values in _days_employed_. 

So let's fill NaN's for retirees with median values to avoid outliers influence.

In [9]:
# 10. let's fill NaN's for retirees with median values to avoid outliers influence
import numpy as np
scoring['days_employed'] = scoring.apply((lambda row: 365004.309916 if np.isnan(row['days_employed']) and row['income_type']=='retiree' else row['days_employed']), axis=1)

retiree_income = scoring[ (scoring['income_type']=='retiree') ]['total_income'].median()
scoring['total_income'] = scoring.apply((lambda row: retiree_income if np.isnan(row['total_income']) and row['income_type']=='retiree' else row['total_income']), axis=1)

In [10]:
# 11. Check that replacement worked
scoring[ (scoring['days_employed']==365004.309916) & (scoring['income_type']=='retiree')].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,365004.309916,65,secondary education,1,civil partnership,1,M,retiree,0,18962.318,to have a wedding
29,0,365004.309916,63,secondary education,1,unmarried,4,F,retiree,0,18962.318,building a real estate
55,0,365004.309916,54,secondary education,1,civil partnership,1,F,retiree,1,18962.318,to have a wedding
67,0,365004.309916,52,bachelor's degree,0,married,0,F,retiree,0,18962.318,purchase of the house for my family
145,0,365004.309916,62,secondary education,1,married,0,M,retiree,0,18962.318,building a property


## Results:

The rest of NaN values will be filled with median values for the related age group and income type. 
For this purpose let's build a pivot table, but first define age categories

In [11]:
# 12. let's build a pivot table, but first define age categories:

def age_category(age):
    if age<18:
        return 'Empty'
    if age>=18 and age<=25:
        return 'From 18 to 25'
    if age>=26 and age<=35:
        return 'From 26 to 35'
    if age>=36 and age<=45:
        return 'From 36 to 45'
    if age>=46 and age<=55:
        return 'From 46 to 55'    
    if age>=56 and age<=65:
        return 'From 56 to 65'    
    if age>=66 and age<=75:
        return 'From 66 to 75'    
    if age>=76 and age<=85:
        return 'From 76 to 85'    
    if age>=86 :
        return 'Above 86'    
scoring['age_category'] = scoring['dob_years'].apply(age_category)


In [12]:
# 13. Build the pivot table for days_employed using age category as index and income type as columns.

days_pivot = scoring[~(scoring['days_employed'].isna()) & (scoring['days_employed']<0)].pivot_table(index=['age_category'], columns='income_type', values='days_employed', aggfunc='median')
days_pivot

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,student
age_category,,,,,,
Empty,-1072.409395,-4945.165547,-1501.480883,NaN,NaN,NaN
From 18 to 25,-749.175677,-1132.739641,-799.243629,NaN,NaN,-578.751554
From 26 to 35,-1266.009797,-1936.232667,-1330.439080,-520.848083,NaN,NaN
From 36 to 45,-1718.022390,-3397.803443,-1795.252264,NaN,-3296.759962,NaN
From 46 to 55,-2021.764755,-3527.162375,-2123.933756,NaN,NaN,NaN
From 56 to 65,-2204.548461,-3956.660670,-2304.455212,NaN,NaN,NaN
From 66 to 75,-2318.709538,-4137.331615,-2830.361431,NaN,NaN,NaN


## Results

Noticed that "Empty" category is non-empty which is strange because usually people below 18 are not eligible for loans.

Replace NaNs according to age + income type grouping with median values and look at the values <18 .

In [13]:
# 14. Replace NaNs according to age + income type grouping with median values and look at the values <18:

scoring['days_employed'] = scoring.apply((lambda row: days_pivot.loc[row['age_category'], row['income_type']] if np.isnan(row['days_employed']) else row['days_employed']), axis=1)
scoring[scoring['dob_years']<18]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

In [14]:
# 15. Check that replacement worked
scoring[(scoring['days_employed'].isna()) ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,From 56 to 65


## Results:

Oops! There was a single record in the category from 56 to 65 and with income_type == entrepreneur, so no reference values are avalable. 

Let's manually replace it using  employee's days from the same age category: -2304.455212.

In [15]:
# 16. Let's manually replace it using  employee's days from the same age category: -2304.455212.
scoring.loc[5936, 'days_employed'] = -2304.455212

In [16]:
# 17. Build the pivot table for total_income using age category as index and income type as columns.

days_pivot = scoring[~(scoring['total_income'].isna()) & (scoring['days_employed']<0)].pivot_table(index=['age_category'], columns='income_type', values='total_income', aggfunc='median')
days_pivot

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,student
age_category,,,,,,
Empty,30532.7365,20090.8960,23198.6255,NaN,NaN,NaN
From 18 to 25,22819.4850,22758.5535,20599.1375,NaN,NaN,15712.26
From 26 to 35,27759.4090,24737.9065,22825.3080,79866.103,NaN,NaN
From 36 to 45,28410.9820,23431.6400,23190.1320,NaN,8612.661,NaN
From 46 to 55,28401.1795,23518.0560,22550.1295,NaN,NaN,NaN
From 56 to 65,28121.5245,24220.7485,23517.5755,NaN,NaN,NaN
From 66 to 75,29314.4045,26089.6870,24643.1985,NaN,NaN,NaN


In [17]:
# 18. Replace NaNs according to age + income type grouping with median values:

scoring['total_income'] = scoring.apply((lambda row: days_pivot.loc[row['age_category'], row['income_type']] if np.isnan(row['total_income']) else row['total_income']), axis=1)

In [18]:
# 19. Check that replacement worked
scoring[(scoring['total_income'].isna()) ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
5936,0,-2304.455212,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,From 56 to 65


In [19]:
# 20. And again entrepreneur withohut references in his group 
# Let's manually replace it using  emploee's income from the same age category: 23517.5755.
scoring.loc[5936, 'total_income'] = 23517.5755

### Conclusion

1. Missing values in *days_employed* and *total_income* columns always occur simultaneously but they do not specifically depend on the columns (age, education, income type or marital status). 
2. From the 1st statement we conclude that the missing values are caused by the absence of customer's work information. Either it was not provided at all (which is unlikely for serious loans) or it was not correctly exported due to technical reasons.
3. To fill in the missing values we looked into typical employment period and income for the corresponding age groups and income types. For example, almost all positive values of *days_employed* refer to the *retiree* income type. As retired people do not provide employment information *days_employed* value remains default which is ~1 000 years (seems like infinity).
4. For types of income other than 'retiree' we built reference pivot_tables to assign median values from each group as a typical value.
5. We've also noticed typos in number of children: 3 records containing '-1' and 9 records containing '20' (there're no other values such as 19, 18 etc. and nowadays 20 children is not a typical value). We should fix those records to 1 and 2 correspondingly. (-1 can also be of a special meaning standing for 'no information' or 'no children' but as there's no possibility to clear it we make an assumption).
6. Age (dob_years) column also has a default value of '0' for some records ('Empty' categorty in the pivot table).

### Data type replacement
First let's fix artifacts for *children* column as discussed in conclusion 5.
Then let's convert float type in *days_employed* and *total_income* columns to integer.
Finally we can fix *days_employed* by taking absolute value for negatives and assigning 0 to positives. We'll lose some information but it's not that important for now.

In [20]:
# fix the number of children -1 -> 1, 20 -> 2 . I use fact that |x|mod 2 maps [-1,20] -> [1, 0] and (2-x) maps it to [1, 2]
scoring['children'] = scoring['children'].apply(lambda child_num: child_num if not (child_num  in [-1,20]) else (2-abs(child_num)%2))

# convert float to int
try:
    scoring['days_employed'] = scoring['days_employed'].astype('int') 
    
except:
    print("Type convertion error of days_employed column!")
try:   
    scoring['total_income'] = scoring['total_income'].astype('int') 
except:
    print("Type convertion error of total_income column!"   )

# fix the days_employed column
scoring['days_employed'] = scoring['days_employed'].apply(lambda days: 0 if days>=0 else abs(days))

# check how we converted    
scoring[['days_employed', 'total_income']].head()


,days_employed,total_income
0,8437,40620
1,4024,17932
2,5623,23341
3,4124,42820
4,0,25378


### Conclusion

1. We fixed the typos in the children column.
2. We improved *days_employment* and *total_income* columns converting their type to integer by using method *astype()* as compared to *to_numeric()* it allows convertation to int.
3. To escape convertation errors method calls were placed into **try-except** block.

### Processing duplicates
Examine several types of duplicates:
1. Use duplicated() method
2. Make string columns low case and look for duplicates once again
3. Consider stemming or lemmatization to group similar categories into one.

In [21]:
# 1.Count the duplicates in general
scoring.duplicated().sum()

54

In [22]:
# 2. Look at the duplicates in general
scoring[scoring.duplicated()].sort_values('total_income').head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
15273,0,0,57,secondary education,1,civil partnership,1,F,retiree,0,18962,wedding ceremony,From 56 to 65
21032,0,0,60,secondary education,1,married,0,F,retiree,0,18962,to become educated,From 56 to 65
20702,0,0,64,secondary education,1,married,0,F,retiree,0,18962,supplementary education,From 56 to 65
20116,0,0,57,secondary education,1,civil partnership,1,M,retiree,0,18962,wedding ceremony,From 56 to 65
19688,0,0,61,secondary education,1,married,0,F,retiree,0,18962,real estate transactions,From 56 to 65


## Results

There're different written forms of the same meaning (upper and lower case strings in education, similar phrases in purpose column).
Let's find unique values in string columns:

In [24]:
# 3. Let's find unique values in string columns:

print('=======================')
print('Education distribution')
print(scoring['education'].value_counts())
print('=======================')
print('Family status distribution')
print(scoring['family_status'].value_counts())
print('=======================')
print('Gender distribution')
print(scoring['gender'].value_counts())
print('=======================')
print('Income type distribution')
print(scoring['income_type'].value_counts())
print('=======================')
print('Purpose distribution')
print(scoring['purpose'].value_counts())
print('=======================')


Education distribution
secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64
Family status distribution
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64
Gender distribution
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
Income type distribution
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur        

## Results

Education column has identical strings but written in different case, so let's turn them into lower case.

In [25]:
# 4. let's turn strings into lower case.


scoring['education'] = scoring['education'].str.lower()
scoring['education'].value_counts()


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

## Results:

The 'purpose' column has a variety of the descriptions for the same thing. Let's make a reduction to the following general  categories: wedding, real estate (property, housing), education (university), car.
Stemming suits here better as there are words from different parts of speech (e.g. education and educated)

Let's find stems of the words and make the common categories 

In [26]:
#    5. Let's find stems of the words and make the common categories 

english_stemmer = SnowballStemmer('english')


def normalize_purpose(name):
    words = nltk.word_tokenize(name)
    stems = [english_stemmer.stem(w) for w in words]
    if 'wed' in stems:
        return 'wedding'
    if 'estat' in stems or 'hous' in stems or 'properti' in stems:
        return 'real estate'
    if 'car' in stems:
        return 'car'
    if 'univers' in stems or 'educ' in stems:
        return 'education'

scoring['purpose'] = scoring['purpose'].apply(normalize_purpose)
scoring['purpose'].value_counts()    


real estate    10840
car             4315
education       4022
wedding         2348
Name: purpose, dtype: int64

After the columns are normalized let's try to find duplicated rows once again!

In [27]:
# 6. After the columns are normalized let's try to find duplicated rows once again!

scoring.duplicated().sum()

405

## Results

The number of duplicates increased almost 8 times. Maybe it was risky to group all loans to one big real estate category.
For now let's drop the duplicates!

In [28]:
# 7. let's drop the duplicates!

scoring.drop_duplicates(inplace=True)
scoring = scoring.reset_index(drop=True)
scoring.duplicated().sum()


0

### Conclusion

1. Found and droped duplicated records. The source of duplicates can occur due to merging records from different databases which contain the same information.
2. duplicated() method is not perfect when there're slightly different values in string columns. To overcome it I made education column low case and grouped all categories with common stems in the 'purpose' column.
3. Other string columns seem not to have duplicates.
4. There's 1 record with 'XNA' value for the gender column! Seems like 'Not available'. Let's omit such a delicate thing...

### Categorizing Data
First let's build dictionaries for education type, family status, income type and purpose to keep the table space.
Earlier we've created the column *'age_category'* to categorize people based on age.
Similarly we can create categories based on work experience in years: *newbie* (0-3), *experienced* (4-9), *old hand* (10+) and categories based on income.

In [29]:
# 1. Build education dictionary

education_dict = scoring[['education', 'education_id']].drop_duplicates()
education_dict = education_dict.reset_index(drop=True)
education_dict

,education,education_id
0,bachelor's degree,0
1,secondary education,1
2,some college,2
3,primary education,3
4,graduate degree,4


In [30]:
# 2. Build family status dictionary

family_status_dict = scoring[['family_status', 'family_status_id']].drop_duplicates()
family_status_dict = family_status_dict.reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,married,0
1,civil partnership,1
2,widow / widower,2
3,divorced,3
4,unmarried,4


In [31]:
# 3. Build income type dictionary

income_type_dict = pd.DataFrame({'income_type':[], 'income_type_id':[]})
income_type_dict['income_type'] = scoring['income_type'].drop_duplicates()
income_type_dict = income_type_dict.reset_index(drop=True)
income_type_dict['income_type_id'] = income_type_dict.index
scoring = scoring.merge(income_type_dict, on='income_type', how='left')
income_type_dict


,income_type,income_type_id
0,employee,0
1,retiree,1
2,business,2
3,civil servant,3
4,unemployed,4
5,entrepreneur,5
6,student,6
7,paternity / maternity leave,7


In [32]:
# 4. Build purpose type dictionary

purpose_type_dict = pd.DataFrame({'purpose':[], 'purpose_id':[]})
purpose_type_dict['purpose'] = scoring['purpose'].drop_duplicates()
purpose_type_dict = purpose_type_dict.reset_index(drop=True)
purpose_type_dict['purpose_id'] = purpose_type_dict.index
scoring = scoring.merge(purpose_type_dict, on='purpose', how='left')
purpose_type_dict

,purpose,purpose_id
0,real estate,0
1,car,1
2,education,2
3,wedding,3


In [33]:
# 5. As soon as we built dictionaries we can drop unnecessary columns:
scoring.drop(columns=['education', 'family_status', 'income_type', 'purpose'], inplace=True)
scoring.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,age_category,income_type_id,purpose_id
0,1,8437,42,0,0,F,0,40620,From 36 to 45,0,0
1,1,4024,36,1,0,F,0,17932,From 36 to 45,0,1
2,0,5623,33,1,0,M,0,23341,From 26 to 35,0,0
3,3,4124,32,1,0,M,0,42820,From 26 to 35,0,2
4,0,0,53,1,1,F,0,25378,From 46 to 55,1,3


In [34]:
# 6. Check by merging with education dictionary:
scoring.merge(education_dict, on='education_id', how='left').head()

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,age_category,income_type_id,purpose_id,education
0,1,8437,42,0,0,F,0,40620,From 36 to 45,0,0,bachelor's degree
1,1,4024,36,1,0,F,0,17932,From 36 to 45,0,1,secondary education
2,0,5623,33,1,0,M,0,23341,From 26 to 35,0,0,secondary education
3,3,4124,32,1,0,M,0,42820,From 26 to 35,0,2,secondary education
4,0,0,53,1,1,F,0,25378,From 46 to 55,1,3,secondary education


In [35]:
# 7. Now let's create work experience category
def work_experience(days):
    if days ==0:
        return 'retired'
    if days/365 <= 3:
        return 'newbie'
    if days/365 <= 9:
        return 'experienced'
    else:
        return 'old hand'

scoring['experience'] = scoring['days_employed'].apply(work_experience)
scoring.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,age_category,income_type_id,purpose_id,experience
0,1,8437,42,0,0,F,0,40620,From 36 to 45,0,0,old hand
1,1,4024,36,1,0,F,0,17932,From 36 to 45,0,1,old hand
2,0,5623,33,1,0,M,0,23341,From 26 to 35,0,0,old hand
3,3,4124,32,1,0,M,0,42820,From 26 to 35,0,2,old hand
4,0,0,53,1,1,F,0,25378,From 46 to 55,1,3,retired


In [45]:
# 8. Let's understand the income range:
scoring.agg({'total_income':['min', 'mean', 'median', 'max']})


,total_income
min,3306.000000
mean,26514.776326
median,23190.000000
max,362496.000000


Let's make 3 categories for income level: 'bronze' for income < 10 000, 'silver' for income from 10 000 to 40 000, 'gold' for income > 40 000

In [47]:
# 9. Let's make 3 categories for income level: 'bronze' for income < 10 000, 'silver' for income from 10 000 to 40 000,
# 'gold' for income > 40 000
def income_level(income):
    if income <=10000:
        return 'bronze'
    if income <= 40000:
        return 'silver'
    else:
        return 'gold'

scoring['income_level'] = scoring['total_income'].apply(income_level)
scoring.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,age_category,income_type_id,purpose_id,experience,income_level
0,1,8437,42,0,0,F,0,40620,From 36 to 45,0,0,old hand,gold
1,1,4024,36,1,0,F,0,17932,From 36 to 45,0,1,old hand,silver
2,0,5623,33,1,0,M,0,23341,From 26 to 35,0,0,old hand,silver
3,3,4124,32,1,0,M,0,42820,From 26 to 35,0,2,old hand,gold
4,0,0,53,1,1,F,0,25378,From 46 to 55,1,3,retired,silver


### Conclusion

1. The data was categorized based on age, work experience and income level. Age groups each of 10 years long were created, and also 3 work experience categories (*newbie* (0-3), *experienced* (4-9), *old hand* (10+)) are based on the number of years of work experience.
2. To classify people based on income we splited them into 3 groups: *bronze* (<=10000), *silver* (10000<x<=40000) and *gold* (>40000).
2. Additionally 4 dictionaries were created to save table space: education, family status, income and purpose dictionaries.

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [71]:
# 1. Let's add a column 'has_kids' to mark records with 0 kids and >0. 
scoring['has_kids'] = scoring.apply(lambda row: 'has kids' if row['children']>0 else 'no kids',axis=1)


In [73]:
# 2. Group by has_kids attribute and calculate probability to have a debt in each group as sum/count:

scoring.groupby('has_kids')['debt'].sum()/scoring.groupby('has_kids')['debt'].count()



has_kids
has kids    0.092991
no kids     0.076723
Name: debt, dtype: float64

To have a more detailed view let's build a pivot table for different dimensions. 

First let's examine gender in relation to having kids.
Agg function sum/count shows the probability to have a debt.

In [74]:
# 3. First let's examine gender in relation to having kids.
#    Agg function sum/count shows the probability to have a debt.

kids_gender_pivot = scoring.pivot_table(index=['gender'], columns='has_kids', values='debt', aggfunc=lambda x: x.sum()/x.count())
kids_gender_pivot


has_kids,has kids,no kids
gender,,
F,0.087372,0.063392
M,0.102602,0.104329
XNA,NaN,0.000000


Next let's examine work experience in relation to having kids.

In [75]:
# 4. Next let's examine work experience in relation to having kids.

kids_experience_pivot = scoring.pivot_table(index=['experience'], columns='has_kids', values='debt', aggfunc=lambda x: x.sum()/x.count())
kids_experience_pivot


has_kids,has kids,no kids
experience,,
experienced,0.089436,0.082376
newbie,0.118878,0.104607
old hand,0.064000,0.052999
retired,0.059748,0.057963


Finally let's examine age in relation to having kids.

In [76]:
# 5. Finally let's examine age in relation to having kids.

kids_age_pivot = scoring.pivot_table(index=['age_category'], columns='has_kids', values='debt', aggfunc=lambda x: x.sum()/x.count())
kids_age_pivot

has_kids,has kids,no kids
age_category,,
Empty,0.093750,0.073529
From 18 to 25,0.105882,0.107588
From 26 to 35,0.107538,0.106117
From 36 to 45,0.086362,0.081618
From 46 to 55,0.075163,0.068194
From 56 to 65,0.057627,0.057828
From 66 to 75,0.057143,0.050228


### Conclusion

1. General and a more detailed view show that there's a relation between having kids and paying a loan on time.
2. People *without kids* tend to pay on time more often than those people *with kids*(debt probability **0.077** is lower than **0.093**).
3. More detailed analysis confirms this idea with some deviations: 
 - For **men** there's *almost no relation* (there's even a slight debt decrease for the men with kids)
 - Young people **from 18 to 25** also show *almost no relation* with a slight debt decrease for people with kids
4. Kids require more resources and money that's why the debt probability increases for people with kids.


- Is there a relation between marital status and repaying a loan on time?

In [88]:
# 6. Group by marital status attribute and calculate probability to have a debt in each group as sum/count:

(scoring.groupby('family_status_id')['debt'].sum()/scoring.groupby('family_status_id')['debt'].count()).rename(family_status_dict['family_status'])

family_status_id
married              0.076929
civil partnership    0.094083
widow / widower      0.066808
divorced             0.071249
unmarried            0.098420
Name: debt, dtype: float64

### Conclusion

1. There's also a relation between marital status and repaying on time: the *highest* probability to have a debt is for **unmarried** people (P = 0.098) and for people in **civil partnership** (P = 0.094)
2. **Married** (P = 0.077) and **divorced** (P = 0.071) have the *medium* probability of having a debt.
3. **Widow / widower** are of the *lowest* probability to have a debt (P = 0.067).
4. The reason for such a pattern is that single people do not have as much support as people who have/had a partner. With a partner the number of social connections increases so the number of people ready to help with repaying the loan increases too.

- Is there a relation between income level and repaying a loan on time?

In [89]:
# 7. Group by income level attribute and calculate probability to have a debt in each group as sum/count:

(scoring.groupby('income_level')['debt'].sum()/scoring.groupby('income_level')['debt'].count())

income_level
bronze    0.062567
gold      0.068990
silver    0.085553
Name: debt, dtype: float64

### Conclusion

1. Surprisingly income level relates to repaying on time but not in a linear manner: people with **small** and **high** income have a comparably *lower* probability of a debt compared to **medium** income people.
2. It can be explaned so that people with small income are given relativly small loans so even in case of difficulties friends and relatives are ready to help, at the same time such people may be afraid to fall into debt hole and they make decisions more thoroughly. High income level allows to take bigger loans but maybe the average loan is not that hard to repay.
3. As soon as income increases from **small** to **medium** people become more confident but at the same time it's harder for them to repay (the loan becomes bigger but the salary is not high enough to face the life difficulties).


- How do different loan purposes affect on-time repayment of the loan?

In [92]:
# 8. Group by purpose_id attribute and calculate probability to have a debt in each group as sum/count:

(scoring.groupby('purpose_id')['debt'].sum()/scoring.groupby('purpose_id')['debt'].count()).rename(purpose_type_dict['purpose'])

purpose_id
real estate    0.073832
car            0.094101
education      0.093340
wedding        0.080659
Name: debt, dtype: float64

### Conclusion

1. The *lowest* debt probability (P = 0.074) is for purposes related to **real estate**.
2. The *medium* debt probability (P = 0.080) is for purposes related to **wedding**.
3. The *highest* debt probability (P = 0.093 and P = 0.094) is for purposes related to **education** and **car**.
4. Real estate loans usually assume guarantees and pledges (the bought property itself) so in case of failure to repay the property can be sold to return the debt. The risk that the property price will fall down is usually low.
5. Wedding loans are not as secure as real estate loans (no pledges such as property) but they are mainly smaller in size than educational and car loans. At the same time people can get gift money from the guests and the relatives are usually concerned into the financial question so risks are lower compared to e.g. car loans.
6. People taking educational loans usually do not have a profession as well as the highly paid job. The car loans are also risky because of road accidents which influence the car's price and health state of the car's owner.

### Step 4. General conclusion

We've studied the data on loans and observed the following:
1. The data had missing values probably caused by technical errors (for example, due to unavailability of the system storing employment information at the moment of data export). We filled the missing data by the typical (median) values for the corresponding groups of people.
2. Besides missing values we've detected several artifacts to investigate with other people:
 - negative values in days_employed column and big positive values for *retired* and *unemployed* people (may be this field is calculated as a difference between employment date and loan request date which results in negatives). Positive values can be interpreted as infinity for people without work.
 - values -1 and 20 in children column. We assumed that these are typos and converted them to 1 and 2.
 - XNA value in the gender column for 1 record. Probably it's not that important.
 - 0 value for the age (dob_years).
3. Data contained the duplicated records which could be caused by data export from different systems having common information. We normalized the dictionary columns and deleted the duplicates.
4. To simplify analysis we categorized quantative data into groups for age, days_employed and income columns and used them in pivot tables to detect patterns.
5. To keep space we created dictionary tables and droped dictionary fields from the main table.
6. Thanks to grouping and pivot tables we investigated the probability of debt for different factors: having kids, marital status, income level and purpose. 
7. Every factor is related to repaying the loan on time.

### Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [X]  conclusions are present on each stage;
- [x]  a general conclusion is made.